In [ ]:
# run this only if you don't have the csv data
!git clone https://github.com/swayable/covid-19-data
!git clone https://github.com/OxCGRT/covid-policy-tracker


Cloning into 'covid-19-data'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 6), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.
Cloning into 'covid-policy-tracker'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 61941 (delta 7), reused 10 (delta 5), pack-reused 61927
Receiving objects: 100% (61941/61941), 4.51 GiB | 28.06 MiB/s, done.
Resolving deltas: 100% (28404/28404), done.
Checking out files: 100% (493/493), done.


In [ ]:
# add import statements here
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
survey = pd.read_csv('/content/covid-19-data/6-15-20.csv')
#survey.drop(columns=['Unnamed: 0', 'CoronavirusEmployment', 'Education_Alt2', 'Ethnicity', 'Gender', 'Religion_Alt1', 'Religiosity_Alt2', 'PoliticalBeliefsNoOp', 'PoliticalPartyNoOp', 'CoronavirusLocalCommunity', 'CoronavirusSupportSystem', 'CoronavirusSymptomSelect', 'CoronavirusWhenShouldReopen', 'CoronavirusUnderstanding4', 'startTime', 'ZipCode'], inplace=True)
survey.dropna(inplace=True)

survey.shape

(16980, 28)

In [ ]:
pd.set_option('display.max_columns', None)
survey.head

<bound method NDFrame.head of        Unnamed: 0  AnnualHouseholdIncome  BirthYear2020  CoronavirusConcern2  \
4000         4000             71000.0000           2002                  8.6   
4004         4004            250000.0000           1990                 10.0   
4012         4012             68000.0000           1993                  6.3   
4016         4016            125000.0000           1980                  8.4   
4017         4017             62000.0000           1998                  8.0   
...           ...                    ...            ...                  ...   
33164       33164              6493.0000           1998                 10.0   
33167       33167              4444.6778           1994                 10.0   
33168       33168            250000.0000           1998                  9.9   
33170       33170            160000.0000           1970                 10.0   
33172       33172             22000.0000           1943                  5.3   

         

In [ ]:
import numpy as np
ref = ['elderly-more-risk', 'corona-all-hospitalize', 'all-spread-covid', 'asymptomatic-spread']
puncs = '''[]','''

x = []
y = []

xWhite = []
xBlack = []
xAsian = []
yWhite = []
yBlack = []
yAsian = []
xHispanic = []
yHispanic = []
xOther = []
yOther = []

xDem = []
yDem = []
xRep = []
yRep = []
xInd = []
yInd = []

for index, row in survey.iterrows():
  cur = []
  cur.append(-(row['BirthYear2020']-2019))
  #cur.append(np.log(row['AnnualHouseholdIncome']))
  cur.append(row['HasCoronavirusBelief'])
  cur.append(row['TrumpApproveNoOp'])
  cur.append(row['Religiosity_Alt2'])
 
  # print(row['CoronavirusConcern2']/10, end=' ')
  # print(row['CoronavirusIntent_Mask']/100, end=' ')
  # print(row['CoronavirusIntent_SixFeet']/100, end=' ')
  # print(row['CoronavirusIntent_StayHome']/100, end=' ')
  # print(row['CoronavirusIntent_WashHands']/100, end=' ')

  # CoronavirusUnderstanding
  s = ''

  items = row['CoronavirusUnderstanding1'] + ' ' + row['CoronavirusUnderstanding2'] + ' ' + row['CoronavirusUnderstanding3']
  for c in items:
    if c not in puncs:
      s += c

  list1 = s.split()
  score = 0

  for elem in list1:
    if elem in ref:
      score += 1
  
  falseOptions = len(list1) - score
  score += (9 - falseOptions)

  #totalScore = row['CoronavirusIntent_Mask']
  totalScore = score + row['CoronavirusConcern2']/10 + row['CoronavirusIntent_Mask']/100 + row['CoronavirusIntent_SixFeet']/100 + row['CoronavirusIntent_StayHome']/100 + row['CoronavirusIntent_WashHands']/100
  #totalScore = row['CoronavirusIntent_Mask']/100
  x.append(cur)
  y.append(totalScore)
  if row['Ethnicity'] == "white":
    yWhite.append(totalScore)
    xWhite.append(cur)
  elif row['Ethnicity'] == "black":
    yBlack.append(totalScore)
    xBlack.append(cur)
  elif row['Ethnicity'] == "asian":
    yAsian.append(totalScore)
    xAsian.append(cur)
  elif row['Ethnicity'] == 'hispanic-latino':
    yHispanic.append(totalScore)
    xHispanic.append(cur)
  elif row['Ethnicity'] == 'other-mixed':
    yOther.append(totalScore)
    xOther.append(cur)
  if row['PoliticalPartyNoOp'] >= 6.5:
    yRep.append(totalScore)
    xRep.append(cur)
  elif row['PoliticalPartyNoOp'] <= 3.5:
    yDem.append(totalScore)
    xDem.append(cur)
  else:
    yInd.append(totalScore)
    xInd.append(cur)

In [1]:
#print(x)
#print(y)

In [2]:
#print(xAsian)
#print(yAsian)

In [ ]:
print(len(xWhite))
print(len(yWhite))
print()
print(len(xBlack))
print(len(yBlack))

9514
9514

3582
3582


In [3]:
#print(xDem)
#print(yDem)

In [ ]:
from sklearn.linear_model import LinearRegression

!pip install pingouin
import pingouin as pg

In [ ]:
lm = pg.linear_regression(x, y)
print(lm)
print()
print("Size of data: " + str(len(y)))

       names       coef        se           T           pval       r2  \
0  Intercept  12.422064  0.064166  193.593260   0.000000e+00  0.19601   
1         x1   0.071849  0.001346   53.361072   0.000000e+00  0.19601   
2         x2  -0.119482  0.007698  -15.521286   5.848004e-54  0.19601   
3         x3  -0.169617  0.005799  -29.248530  1.625565e-183  0.19601   
4         x4   0.017773  0.006460    2.751132   5.945272e-03  0.19601   

    adj_r2   CI[2.5%]  CI[97.5%]  
0  0.19582  12.296292  12.547835  
1  0.19582   0.069209   0.074488  
2  0.19582  -0.134571  -0.104393  
3  0.19582  -0.180984  -0.158250  
4  0.19582   0.005110   0.030436  

Size of data: 16980


In [ ]:
lm = pg.linear_regression(xWhite, yWhite)
print(lm)
print()
print("Size of data: " + str(len(yWhite)))

       names       coef        se           T           pval        r2  \
0  Intercept  13.792169  0.082557  167.062294   0.000000e+00  0.166938   
1         x1   0.053225  0.001665   31.965987  4.610898e-213  0.166938   
2         x2  -0.094129  0.010955   -8.592005   9.893739e-18  0.166938   
3         x3  -0.194940  0.007175  -27.168467  1.312657e-156  0.166938   
4         x4  -0.004481  0.008057   -0.556097   5.781579e-01  0.166938   

     adj_r2   CI[2.5%]  CI[97.5%]  
0  0.166588  13.630340  13.953999  
1  0.166588   0.049961   0.056489  
2  0.166588  -0.115604  -0.072654  
3  0.166588  -0.209005  -0.180875  
4  0.166588  -0.020274   0.011313  

Size of data: 9514


In [ ]:
lm = pg.linear_regression(xBlack, yBlack)
print(lm)
print()
print("Size of data: " + str(len(yBlack)))

       names       coef        se          T           pval       r2  \
0  Intercept  10.036839  0.141847  70.758233   0.000000e+00  0.26365   
1         x1   0.081763  0.003421  23.901566  2.921397e-117  0.26365   
2         x2  -0.097163  0.014940  -6.503350   8.944303e-11  0.26365   
3         x3  -0.172659  0.014096 -12.248916   8.030595e-34  0.26365   
4         x4   0.201454  0.014652  13.749275   5.894523e-42  0.26365   

     adj_r2  CI[2.5%]  CI[97.5%]  
0  0.262826  9.758730  10.314948  
1  0.262826  0.075056   0.088470  
2  0.262826 -0.126455  -0.067870  
3  0.262826 -0.200295  -0.145022  
4  0.262826  0.172727   0.230181  

Size of data: 3582


In [ ]:
lm = pg.linear_regression(xAsian, yAsian)
print(lm)
print()
print("Size of data: " + str(len(yAsian)))

       names       coef        se          T           pval       r2  \
0  Intercept  13.844358  0.312014  44.370893  1.860913e-218  0.17004   
1         x1   0.043252  0.007449   5.806738   9.165190e-09  0.17004   
2         x2  -0.167321  0.031790  -5.263401   1.815242e-07  0.17004   
3         x3  -0.244022  0.028666  -8.512664   8.268507e-17  0.17004   
4         x4   0.005079  0.030528   0.166376   8.679030e-01  0.17004   

     adj_r2   CI[2.5%]  CI[97.5%]  
0  0.165916  13.231900  14.456816  
1  0.165916   0.028631   0.057872  
2  0.165916  -0.229721  -0.104921  
3  0.165916  -0.300291  -0.187754  
4  0.165916  -0.054845   0.065003  

Size of data: 810


In [ ]:
lm = pg.linear_regression(xHispanic, yHispanic)
print(lm)
print()
print("Size of data: " + str(len(yHispanic)))

       names       coef        se          T          pval        r2  \
0  Intercept  12.037992  0.196374  61.301428  0.000000e+00  0.164681   
1         x1   0.064167  0.004952  12.957445  5.349954e-37  0.164681   
2         x2  -0.128055  0.020534  -6.236380  5.397776e-10  0.164681   
3         x3  -0.217925  0.018926 -11.514335  8.566804e-30  0.164681   
4         x4   0.095159  0.020569   4.626357  3.948090e-06  0.164681   

     adj_r2   CI[2.5%]  CI[97.5%]  
0  0.163092  11.652885  12.423099  
1  0.163092   0.054456   0.073879  
2  0.163092  -0.168323  -0.087787  
3  0.163092  -0.255041  -0.180808  
4  0.163092   0.054821   0.135496  

Size of data: 2107


In [ ]:
lm = pg.linear_regression(xOther, yOther)
print(lm)
print()
print("Size of data: " + str(len(yOther)))

       names       coef        se          T           pval        r2  \
0  Intercept  11.849838  0.290099  40.847588  2.374660e-212  0.106728   
1         x1   0.048142  0.006471   7.439236   2.241232e-13  0.106728   
2         x2  -0.093955  0.035439  -2.651136   8.154015e-03  0.106728   
3         x3  -0.175858  0.029431  -5.975293   3.231107e-09  0.106728   
4         x4   0.129476  0.030282   4.275710   2.095181e-05  0.106728   

     adj_r2   CI[2.5%]  CI[97.5%]  
0  0.103014  11.280538  12.419137  
1  0.103014   0.035442   0.060842  
2  0.103014  -0.163502  -0.024407  
3  0.103014  -0.233615  -0.118102  
4  0.103014   0.070050   0.188902  

Size of data: 967


In [ ]:
lm = pg.linear_regression(xDem, yDem)
print(lm)
print()
print("Size of data: " + str(len(yDem)))

       names       coef        se           T           pval        r2  \
0  Intercept  12.851671  0.115746  111.033393   0.000000e+00  0.197273   
1         x1   0.066093  0.002537   26.051601  1.357740e-140  0.197273   
2         x2  -0.039909  0.016290   -2.449936   1.432146e-02  0.197273   
3         x3  -0.289875  0.015269  -18.984268   1.028699e-77  0.197273   
4         x4   0.010496  0.011146    0.941626   3.464285e-01  0.197273   

     adj_r2   CI[2.5%]  CI[97.5%]  
0  0.196645  12.624759  13.078583  
1  0.196645   0.061119   0.071066  
2  0.196645  -0.071844  -0.007974  
3  0.196645  -0.319809  -0.259941  
4  0.196645  -0.011356   0.032347  

Size of data: 5117


In [ ]:
lm = pg.linear_regression(xRep, yRep)
print(lm)
print()
print("Size of data: " + str(len(yRep)))

       names       coef        se          T           pval        r2  \
0  Intercept  11.019274  0.139177  79.174777   0.000000e+00  0.203969   
1         x1   0.063226  0.002305  27.433382  3.208944e-154  0.203969   
2         x2  -0.137239  0.010993 -12.484641   3.110006e-35  0.203969   
3         x3   0.000109  0.011803   0.009256   9.926150e-01  0.203969   
4         x4   0.076933  0.012817   6.002475   2.083784e-09  0.203969   

     adj_r2   CI[2.5%]  CI[97.5%]  
0  0.203316  10.746425  11.292123  
1  0.203316   0.058707   0.067744  
2  0.203316  -0.158789  -0.115688  
3  0.203316  -0.023030   0.023248  
4  0.203316   0.051806   0.102059  

Size of data: 4887


In [ ]:
lm = pg.linear_regression(xInd, yInd)
print(lm)
print()
print("Size of data: " + str(len(yInd)))

In [ ]:
x = np.arange(5, 95, 10)
print(x)

[ 5 15 25 35 45 55 65 75 85]


In [ ]:
yCovidDeath = np.array([49980, 438, 23283,183880,144085,132457, 125256, 120766, 114101])
print(yCovidDeath)

[ 49980    438  23283 183880 144085 132457 125256 120766 114101]


In [ ]:
yPercentCovidDeath = np.array([7.09, 0.06, 2.40, 18.76, 13.15,10.49, 10.23,8.37,8.80])
print(yPercentCovidDeath)

[ 7.09  0.06  2.4  18.76 13.15 10.49 10.23  8.37  8.8 ]


In [ ]:
lm = pg.linear_regression(x, yCovidDeath)
print(lm)

       names          coef            se         T      pval        r2  \
0  Intercept  41611.341667  35546.115490  1.170630  0.280052  0.333865   
1         x1   1283.318333    685.143449  1.873065  0.103212  0.333865   

     adj_r2      CI[2.5%]      CI[97.5%]  
0  0.238702 -42441.865051  125664.548384  
1  0.238702   -336.788481    2903.425148  


In [ ]:
lm = pg.linear_regression(x, yPercentCovidDeath)
print(lm)

       names      coef        se         T      pval        r2    adj_r2  \
0  Intercept  5.879667  3.737105  1.573321  0.159643  0.104972 -0.022889   
1         x1  0.065267  0.072032  0.906080  0.395011  0.104972 -0.022889   

   CI[2.5%]  CI[97.5%]  
0 -2.957183  14.716516  
1 -0.105062   0.235595  
